In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [2]:
data = np.zeros([731, 5])

In [3]:
f = open('datat.log')
row = -1
col = 0
for line in f.readlines():
    string = line.split(":")
    if len(string) != 1:
        data[row, col] = string[1]
        col = col + 1
    else:
        row = row + 1
        col = 0

In [4]:
data

array([[ 72.,   2.,  51., -58.,  45.],
       [ 72.,   2., 101., -57.,  45.],
       [ 72.,   2., 151., -57.,  45.],
       ...,
       [378.,   4., 201., -45.,  46.],
       [378.,   4., 251., -41.,  47.],
       [378.,   4.,   1., -42.,  45.]])

In [5]:
data = pd.DataFrame(data, columns=["tx_data", "sender", "next_freq", "rssi", "lqi"])
data

,tx_data,sender,next_freq,rssi,lqi
0,72.0,2.0,51.0,-58.0,45.0
1,72.0,2.0,101.0,-57.0,45.0
2,72.0,2.0,151.0,-57.0,45.0
3,72.0,2.0,201.0,-52.0,45.0
4,72.0,2.0,251.0,-51.0,45.0
...,...,...,...,...,...
726,378.0,4.0,101.0,-45.0,47.0
727,378.0,4.0,151.0,-47.0,47.0
728,378.0,4.0,201.0,-45.0,46.0
729,378.0,4.0,251.0,-41.0,47.0


In [6]:
data['tx_data'].value_counts()

378.0    353
72.0     289
377.0     89
Name: tx_data, dtype: int64

In [7]:
data.iloc[:,0][data.iloc[:,0] == 72.0] = 1
data.iloc[:,0][data.iloc[:,0] == 377.0] = 2
data.iloc[:,0][data.iloc[:,0] == 378.0] = 2

In [8]:
data['diff'] = data.sender.diff()
data.iloc[0,-1] = -1

In [9]:
data

,tx_data,sender,next_freq,rssi,lqi,diff
0,1.0,2.0,51.0,-58.0,45.0,-1.0
1,1.0,2.0,101.0,-57.0,45.0,0.0
2,1.0,2.0,151.0,-57.0,45.0,0.0
3,1.0,2.0,201.0,-52.0,45.0,0.0
4,1.0,2.0,251.0,-51.0,45.0,0.0
...,...,...,...,...,...,...
726,2.0,4.0,101.0,-45.0,47.0,0.0
727,2.0,4.0,151.0,-47.0,47.0,0.0
728,2.0,4.0,201.0,-45.0,46.0,0.0
729,2.0,4.0,251.0,-41.0,47.0,0.0


In [10]:
data['diff'].value_counts()

 0.0    605
 1.0     84
-2.0     41
-1.0      1
Name: diff, dtype: int64

In [11]:
stacked_data = np.zeros([data['diff'].value_counts()[-2]*3+3, 8])

In [12]:
sender = 0
row_num = -1
for index,row in data.iterrows():
    if row['diff'] != 0:
        sender = row.sender
        row_num += 1
    stacked_data[row_num, 0] = row.tx_data
    stacked_data[row_num, 1] = sender
    if row.next_freq == 51.0:
        stacked_data[row_num, 2] = row.rssi
    elif row.next_freq == 101.0:
        stacked_data[row_num, 3] = row.rssi
    elif row.next_freq == 151.0:
        stacked_data[row_num, 4] = row.rssi
    elif row.next_freq == 201.0:
        stacked_data[row_num, 5] = row.rssi
    elif row.next_freq == 251.0:
        stacked_data[row_num, 6] = row.rssi
    elif row.next_freq == 1.0:
        stacked_data[row_num, 7] = row.rssi

In [13]:
stacked_data = stacked_data[:,:-1]

In [14]:
stacked_data

array([[  1.,   2., -58., -57., -57., -52., -51.],
       [  1.,   3., -62., -58., -55., -56., -56.],
       [  1.,   4., -57., -53., -49., -47., -49.],
       [  1.,   2., -41., -45., -42., -43., -43.],
       [  1.,   3., -48., -48., -52., -68., -56.],
       [  1.,   4., -44., -46., -47., -50., -50.],
       [  1.,   2., -44., -44., -42., -41., -42.],
       [  1.,   3., -67., -64., -57., -52., -51.],
       [  1.,   4., -55., -53., -53., -56., -52.],
       [  1.,   2., -52., -52., -52., -50., -52.],
       [  1.,   3., -57., -57., -58., -64., -76.],
       [  1.,   4., -60., -55., -59., -56., -57.],
       [  1.,   2., -47., -48., -49., -48., -53.],
       [  1.,   3., -56., -62., -71., -64., -79.],
       [  1.,   4., -59., -61., -59., -55., -61.],
       [  1.,   2., -59., -64., -80., -68., -60.],
       [  1.,   3., -54., -53., -56., -59., -71.],
       [  1.,   4., -57., -52., -48., -46., -48.],
       [  1.,   2., -51., -54., -58., -54., -53.],
       [  1.,   3., -62., -58.,

In [15]:
stacked_data2 = np.zeros([int(stacked_data.shape[0]/3*5), 5])

In [16]:
stacked_data2.shape

(210, 5)

In [17]:
data_row = 0
row_num = 0
while row_num < stacked_data.shape[0]:
    col_num = 2
    while col_num < 7:
        stacked_data2[data_row, 0] = stacked_data[row_num, 0]
        stacked_data2[data_row, 1] = col_num-1
        stacked_data2[data_row, 2] = stacked_data[row_num, col_num]
        stacked_data2[data_row, 3] = stacked_data[row_num+1, col_num]
        stacked_data2[data_row, 4] = stacked_data[row_num+2, col_num]
        col_num += 1
        data_row += 1
    row_num += 3

In [18]:
stacked_data2

array([[  1.,   1., -58., -62., -57.],
       [  1.,   2., -57., -58., -53.],
       [  1.,   3., -57., -55., -49.],
       ...,
       [  2.,   3., -39., -44., -47.],
       [  2.,   4., -37., -42., -45.],
       [  2.,   5., -35., -41., -41.]])

In [19]:
np.savetxt("result.csv", stacked_data2, delimiter=",")

In [20]:
stacked_data2_frame = pd.DataFrame(stacked_data2)
stacked_data2_frame

,0,1,2,3,4
0,1.0,1.0,-58.0,-62.0,-57.0
1,1.0,2.0,-57.0,-58.0,-53.0
2,1.0,3.0,-57.0,-55.0,-49.0
3,1.0,4.0,-52.0,-56.0,-47.0
4,1.0,5.0,-51.0,-56.0,-49.0
...,...,...,...,...,...
205,2.0,1.0,-41.0,-51.0,-43.0
206,2.0,2.0,-40.0,-46.0,-45.0
207,2.0,3.0,-39.0,-44.0,-47.0
208,2.0,4.0,-37.0,-42.0,-45.0


In [21]:
freq1 = stacked_data2_frame[stacked_data2_frame[1] == 1]
freq2 = stacked_data2_frame[stacked_data2_frame[1] == 2]
freq3 = stacked_data2_frame[stacked_data2_frame[1] == 3]
freq4 = stacked_data2_frame[stacked_data2_frame[1] == 4]

In [36]:
final_freq = pd.concat([freq1.reset_index(), freq2.reset_index(), freq3.reset_index(), freq4.reset_index()], axis=1)
final_freq = final_freq.iloc[:,[1,3,4,5,9,10,11,15,16,17,21,22,23]]

In [22]:
final_freq = pd.concat([freq1.reset_index(), freq4.reset_index()], axis=1)
final_freq = final_freq.iloc[:,[1,3,4,5,9,10,11]]

In [23]:
X = np.asarray(final_freq.iloc[:,1:])
y = np.asarray(final_freq.iloc[:,0])

In [25]:
X

array([[-58., -62., -57., -52., -56., -47.],
       [-41., -48., -44., -43., -68., -50.],
       [-44., -67., -55., -41., -52., -56.],
       [-52., -57., -60., -50., -64., -56.],
       [-47., -56., -59., -48., -64., -55.],
       [-59., -54., -57., -68., -59., -46.],
       [-51., -62., -53., -54., -56., -66.],
       [-45., -59., -41., -48., -57., -42.],
       [-49., -56., -44., -52., -53., -45.],
       [-65., -56., -46., -52., -55., -45.],
       [-44., -49., -44., -49., -51., -47.],
       [-59., -53., -51., -59., -58., -53.],
       [-49., -52., -60., -49., -47., -68.],
       [-52., -56., -56., -52., -60., -49.],
       [-47., -53., -52., -47., -58., -59.],
       [-63., -48., -51., -64., -46., -42.],
       [-58., -55., -58., -56., -49., -58.],
       [-52., -41., -49., -52., -36., -45.],
       [-59., -45., -55., -52., -40., -55.],
       [-64., -48., -51., -74., -39., -47.],
       [-55., -45., -64., -51., -49., -51.],
       [-47., -37., -38., -46., -32., -37.],
       [-4

In [26]:
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)

In [27]:
scaler.mean_

array([-51.69047619, -49.26190476, -47.23809524, -51.21428571,
       -48.33333333, -46.11904762])

In [28]:
scaler.scale_

array([ 6.48620549,  7.00700168,  7.7699345 ,  8.70012511, 10.04671628,
        7.9498499 ])

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, test_size=0.3)

In [48]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

In [49]:
clf.score(X_scaled, y)

0.9285714285714286

In [50]:
clf.score(X_test, y_test)

0.8461538461538461

In [51]:
clf.coef_

array([[-0.33613883,  1.53720366,  0.40401378,  0.64988429,  1.01556114,
         0.7508203 ]])

In [34]:
clf.intercept_

array([0.61368275])

In [35]:
clf.predict_proba(X_test)

array([[0.15268401, 0.84731599],
       [0.27871853, 0.72128147],
       [0.98459586, 0.01540414],
       [0.02906974, 0.97093026],
       [0.0254735 , 0.9745265 ],
       [0.37965043, 0.62034957],
       [0.41672219, 0.58327781],
       [0.9292975 , 0.0707025 ],
       [0.99025028, 0.00974972],
       [0.5164329 , 0.4835671 ],
       [0.00158946, 0.99841054],
       [0.18758351, 0.81241649],
       [0.98018872, 0.01981128]])

In [99]:
y_test

array([1., 2., 2., 2., 1., 2., 1., 1., 2., 2., 1., 2., 2.])

In [91]:
X_test[0,:]

array([ 0.41479972, -0.3907656 , -1.64247263,  0.25451194,  0.13271335,
       -2.75237302])

In [89]:
clf.coef_

array([[-0.44268162,  1.38089719,  0.27767763,  0.56391309,  0.87230852,
         0.60084217]])

In [77]:
temp = np.array([-54, -50, -48, -51, -59, -68])

In [78]:
temp

array([-54, -50, -48, -51, -59, -68])

In [79]:
temp = scaler.transform(temp[np.newaxis,:])

In [80]:
temp

array([[-0.35606701, -0.10533681, -0.09805807,  0.02463019, -1.06170677,
        -2.75237302]])

In [81]:
temp_coef = np.array([-0.4427, 1.3809, 0.2777, 0.5639, 0.8723, 0.6008])
temp_inter = 0.8034

In [82]:
1/(1+np.exp(-1*(np.sum(temp*temp_coef)+temp_inter)))

0.14460925940174876

In [98]:
1/(1+np.exp(-1*(np.sum(X_test[0,:]*clf.coef_)+clf.intercept_)))

array([0.14549339])

In [97]:
y_test

array([1., 2., 2., 2., 1., 2., 1., 1., 2., 2., 1., 2., 2.])

In [49]:
np.savetxt("freq1.csv", np.concatenate([y[:,np.newaxis],X_scaled],axis=1), delimiter=",")

In [83]:
from numpy import genfromtxt
my_data = genfromtxt('../data/data_recollect.log', delimiter=',')

In [84]:
my_data

array([[  1., -54., -47., -42., -39., -52., -51.],
       [  1., -45., -40., -47., -50., -55., -52.],
       [  1., -47., -43., -50., -47., -52., -45.],
       [  1., -51., -52., -51., -51., -47., -45.],
       [  1., -52., -49., -46., -42., -58., -59.],
       [  1., -52., -59., -53., -52., -53., -59.],
       [  1., -44., -41., -41., -41., -52., -50.],
       [  1., -58., -49., -45., -40., -57., -47.],
       [  1., -52., -48., -49., -45., -62., -52.],
       [  1., -47., -45., -59., -46., -53., -54.],
       [  1., -56., -52., -50., -46., -51., -53.],
       [  1., -56., -62., -49., -40., -52., -51.],
       [  1., -52., -41., -46., -45., -56., -54.],
       [  1., -54., -42., -47., -44., -49., -47.],
       [  1., -47., -47., -49., -47., -43., -39.],
       [  1., -46., -40., -50., -47., -54., -54.],
       [  1., -50., -50., -55., -47., -62., -61.],
       [  1., -47., -56., -51., -50., -56., -57.],
       [  1., -50., -52., -54., -47., -66., -59.],
       [  1., -47., -56., -50.,

In [90]:
X = np.asarray(my_data[:,1:])
y = np.asarray(my_data[:,0])

In [91]:
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)

In [134]:
def bmatrix(a):
    """Returns a LaTeX bmatrix

    :a: numpy array
    :returns: LaTeX bmatrix as a string
    """
    if len(a.shape) > 2:
        raise ValueError('bmatrix can at most display two dimensions')
    temp_string = np.array2string(a, formatter={'float_kind':lambda x: "{:.4f}".format(x)})
    lines = temp_string.replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{bmatrix}']
    rv += ['  ' + ', '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{bmatrix}']
    return '\n'.join(rv)

In [92]:
scaler.mean_

array([-49.12698413, -45.82539683, -45.77777778, -42.63492063,
       -51.38095238, -49.06349206])

In [93]:
scaler.scale_

array([7.13696406, 6.54056859, 5.99323546, 4.79517478, 6.51841974,
       6.92332844])

In [136]:
bmatrix(scaler.scale_)

'\\begin{bmatrix}\n  7.1370, 6.5406, 5.9932, 4.7952, 6.5184, 6.9233\\\\\n\\end{bmatrix}'

In [ ]:
bmatrix(scaler.mean_)

In [113]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, test_size=0.2)

In [117]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

In [118]:
clf.score(X_scaled, y)

0.8412698412698413

In [119]:
clf.score(X_test, y_test)

0.9230769230769231

In [120]:
clf.coef_

array([[ 0.67916347, -0.05254733,  0.47520648,  0.59513537,  0.66723814,
         0.51040948]])

In [121]:
clf.intercept_

array([-0.02472834])

In [137]:
bmatrix(clf.coef_)

'\\begin{bmatrix}\n  0.6792, -0.0525, 0.4752, 0.5951, 0.6672, 0.5104\\\\\n\\end{bmatrix}'

In [138]:
bmatrix(clf.intercept_)

'\\begin{bmatrix}\n  -0.0247\\\\\n\\end{bmatrix}'

In [122]:
clf.predict_proba(X_test)

array([[0.78189465, 0.21810535],
       [0.83655422, 0.16344578],
       [0.48499283, 0.51500717],
       [0.14828111, 0.85171889],
       [0.73606435, 0.26393565],
       [0.57463656, 0.42536344],
       [0.8117365 , 0.1882635 ],
       [0.89408436, 0.10591564],
       [0.8350063 , 0.1649937 ],
       [0.06367802, 0.93632198],
       [0.37358245, 0.62641755],
       [0.54446134, 0.45553866],
       [0.01115885, 0.98884115]])

In [123]:
y_test

array([1., 1., 2., 2., 2., 1., 1., 1., 1., 2., 2., 1., 2.])